In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random

In [ ]:
BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/dacon_creditcard/open/'

train = pd.read_csv(BASE_DIR+'train.csv')
train = train.drop(['index'], axis=1)

test = pd.read_csv(BASE_DIR+'test.csv')
test = test.drop(['index'], axis=1)

submit = pd.read_csv(BASE_DIR+'sample_submission.csv')

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       8171
family_size         0
begin_month         0
credit              0
dtype: int64
gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       3152
family_size         0
begin_month         0
dtype: int64


In [ ]:
# fill Null Values
train.fillna('X', inplace=True) 
test.fillna('X', inplace=True)

In [ ]:
# 데이터 전처리
## Child_num
train['child_num']=np.where(train['child_num'] >= 1,
                             'N', # 0이면 자녀가 없음
                             'Y') # 1이상이면 자녀가 있음으로 나뉨
test['child_num']=np.where(test['child_num'] >= 1,'N','Y')

## income_total
#train['income_total']= round(train['income_total']/10000)
#test['income_total']= round(test['income_total']/10000)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         26457 non-null  object 
 1   car            26457 non-null  object 
 2   reality        26457 non-null  object 
 3   child_num      26457 non-null  object 
 4   income_total   26457 non-null  float64
 5   income_type    26457 non-null  object 
 6   edu_type       26457 non-null  object 
 7   family_type    26457 non-null  object 
 8   house_type     26457 non-null  object 
 9   DAYS_BIRTH     26457 non-null  int64  
 10  DAYS_EMPLOYED  26457 non-null  int64  
 11  FLAG_MOBIL     26457 non-null  int64  
 12  work_phone     26457 non-null  int64  
 13  phone          26457 non-null  int64  
 14  email          26457 non-null  int64  
 15  occyp_type     26457 non-null  object 
 16  family_size    26457 non-null  float64
 17  begin_month    26457 non-null  float64
 18  credit

In [ ]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [ ]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])

train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [ ]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [ ]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1100,min_samples_split=2, min_samples_leaf=4)
   # min_samples_split=2, min_samples_leaf=4, max_features=None,loss ='ls'
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=100,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.678699	valid_1's multi_logloss: 0.758326
[200]	training's multi_logloss: 0.604786	valid_1's multi_logloss: 0.741679
[300]	training's multi_logloss: 0.550319	valid_1's multi_logloss: 0.734197
[400]	training's multi_logloss: 0.503597	valid_1's multi_logloss: 0.731206
[500]	training's multi_logloss: 0.465041	valid_1's multi_logloss: 0.73119
Early stopping, best iteration is:
[418]	training's multi_logloss: 0.49575	valid_1's multi_logloss: 0.730827


====================================2============================================
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.67605	valid_1's multi_logloss: 0.769666
[200]	training's multi_logloss: 0.602036	valid_1's multi_logloss: 0.752958
[300]	training's multi_logloss: 0.546123	valid_1's multi_logloss: 0.747878
[

In [ ]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [ ]:
submit.to_csv(BASE_DIR + '20210513_test_submit_lightgbm.csv', index=False) # 0.7272812144